## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-18-20-52-56 +0000,nypost,Aussie PM blasted as having ‘blood on his hand...,https://nypost.com/2025/12/18/world-news/bondi...
1,2025-12-18-20-51-39 +0000,nyt,7 Dead After Plane Registered to NASCAR Driver...,https://www.nytimes.com/2025/12/18/us/statesvi...
2,2025-12-18-20-51-00 +0000,nyt,Authorities Search for a New Person of Interes...,https://www.nytimes.com/2025/12/18/us/brown-mi...
3,2025-12-18-20-50-25 +0000,nypost,"Trump’s $1,776 ‘warrior dividend’ to service m...",https://nypost.com/2025/12/18/us-news/presiden...
4,2025-12-18-20-47-00 +0000,wsj,"Hogan Lovells and Cadwalader to Merge, Creatin...",https://www.wsj.com/business/deals/hogan-lovel...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2358/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
28,trump,61
23,new,24
7,bondi,15
42,billion,13
157,he,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
367,2025-12-17-21-58-00 +0000,wsj,President Trump on Tuesday received and discus...,https://www.wsj.com/politics/elections/trump-t...,155
60,2025-12-18-19-01-29 +0000,nyt,"Trump Media to Merge With TAE Technologies, a ...",https://www.nytimes.com/2025/12/18/business/tr...,113
263,2025-12-18-06-03-00 +0000,wsj,President Trump defended his handling of the e...,https://www.wsj.com/politics/policy/trump-defe...,112
33,2025-12-18-20-03-04 +0000,bbc,Kennedy Center to be renamed Trump-Kennedy Cen...,https://www.bbc.com/news/articles/cp84pxvp87eo...,111
118,2025-12-18-17-02-00 +0000,wsj,Trump Media to Merge With Nuclear Fusion Firm ...,https://www.wsj.com/business/deals/trump-media...,110


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
367,155,2025-12-17-21-58-00 +0000,wsj,President Trump on Tuesday received and discus...,https://www.wsj.com/politics/elections/trump-t...
209,74,2025-12-18-11-43-19 +0000,nypost,Mourners grieve 10-year-old slain in Bondi mas...,https://nypost.com/2025/12/18/world-news/mourn...
316,59,2025-12-18-00-21-00 +0000,wsj,Dan Bongino said that he is stepping down as d...,https://www.wsj.com/politics/policy/fbi-deputy...
155,55,2025-12-18-14-55-00 +0000,wsj,The Bank of England cut its key interest rate ...,https://www.wsj.com/economy/central-banking/ba...
60,52,2025-12-18-19-01-29 +0000,nyt,"Trump Media to Merge With TAE Technologies, a ...",https://www.nytimes.com/2025/12/18/business/tr...
33,50,2025-12-18-20-03-04 +0000,bbc,Kennedy Center to be renamed Trump-Kennedy Cen...,https://www.bbc.com/news/articles/cp84pxvp87eo...
341,47,2025-12-17-23-06-12 +0000,nypost,Senate approves bill inspired by DC plane cras...,https://nypost.com/2025/12/17/us-news/senate-a...
114,45,2025-12-18-17-05-01 +0000,nypost,"NASCAR star Greg Biffle was flying with wife, ...",https://nypost.com/2025/12/18/sports/private-j...
347,40,2025-12-17-22-45-06 +0000,bbc,Rob Reiner's family breaks silence as son Nick...,https://www.bbc.com/news/articles/cx239pp7gl7o...
220,39,2025-12-18-11-00-00 +0000,latimes,Want to walk your loved ones right up to the a...,https://www.latimes.com/california/story/2025-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
